# [실습] OpenAI API를 이용하여 뉴스 기사 정리 봇 만들기

OpenAI Chat API를 이용해 뉴스 검색 결과를 정리하는 프로그램을 만들어 보겠습니다.


네이버의 뉴스 검색 API를 사용하겠습니다.   
(https://developers.naver.com/apps/#/register?defaultScope=search)

In [1]:
!pip install openai tiktoken --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached tiktoken-0.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.7.0
    Uninstalling tiktoken-0.7.0:
      Successfully uninstalled tiktoken-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.113 requires tiktoken<0.8.0,>=0.7.0, but you have tiktoken 0.8.0 which is incompatible.


In [1]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
openai_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# os의 환경 변수에 API 키 복사 붙여넣기
import openai
import os

# OPENAI API KEY 설정
os.environ['OPENAI_API_KEY']=openai_api_key

client = openai.OpenAI()


assert len(os.environ['OPENAI_API_KEY']) > 0, "OPENAI_API_KEY가 환경 변수에 설정되어 있지 않습니다. API 키를 설정해주세요."

# API 키가 설정되어 있다면, 이 지점 이후의 코드가 실행됩니다.
print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")

OPENAI_API_KEY가 정상적으로 설정되어 있습니다.


## 1. LLM으로 웹 크롤링 코드 구현 프롬프트 만들기

네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성하겠습니다.   
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 전달하는 함수를 요청합니다.

**네이버 뉴스 API를 사용한다는 내용을 명시하면 정확도가 올라갑니다.**

In [4]:
prompt = '''
네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성해 주세요.
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 아래 포맷의 문자열로 변환하는 함수 get_news(query) 를 만들어 주세요.
---
제목: (뉴스 제목)
URL: (뉴스 링크- 있으면)
내용: (뉴스 내용)
---

결과는 result에 저장하고, 출력하세요.
---




# 검색어
query = '생성형 AI'


# 검색 결과를 return하는 함수
def get_news(query):


'''



In [5]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [{'role':'system', 'content':'당신은 파이썬 코딩의 전문가입니다. 설명 없이 코드만 출력하세요.'},
          {'role':'user', 'content': prompt}],

    temperature =  0,
    max_tokens = 2000

)
print(response.choices[0].message.content)

```python
import requests

def get_news(query):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=30&sort=sim"
    headers = {
        'X-Naver-Client-Id': 'YOUR_CLIENT_ID',
        'X-Naver-Client-Secret': 'YOUR_CLIENT_SECRET'
    }
    
    response = requests.get(url, headers=headers)
    news_data = response.json()
    
    result = ""
    for item in news_data['items']:
        result += f"제목: {item['title']}\n"
        result += f"URL: {item['link']}\n"
        result += f"내용: {item['description']}\n"
        result += "---\n"
    
    return result

# 검색어
query = '생성형 AI'
result = get_news(query)
print(result)
```


In [ ]:
# VS CODE에서 위 결과가 truncated 되는 경우 scrollable element를 클릭하여 풀 코드 확인

아래에 전체 코드를 넣고 실행합니다.

In [9]:
import requests

def get_news(query):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=30&sort=sim"
    headers = {
        'X-Naver-Client-Id': os.getenv("NAVER_CLIENT_ID"),
        'X-Naver-Client-Secret': os.getenv("NAVER_CLIENT_SECRET")
    }
    
    response = requests.get(url, headers=headers)
    news_items = response.json().get('items', [])
    
    result = ""
    for item in news_items:
        title = item['title']
        link = item['link']
        description = item['description']
        
        result += f"제목: {title}\n"
        result += f"URL: {link}\n"
        result += f"내용: {description}\n"
        result += "---\n"
    
    print(result)
    return result

# 검색어
query = '생성형 AI'
get_news(query)


제목: 이재용-올트먼-손정의 ‘한미일 <b>AI</b> 동맹’ 손잡았다
URL: https://n.news.naver.com/mnews/article/020/0003613234?sid=101
내용: 업계에서는 오픈AI와 엔비디아 등 미국 빅테크 주도의 <b>생성형 AI</b>와 AI 반도체, 일본 소프트뱅크·ARM의 자금 조달과 칩 설계 기술에 한국 기업이 갖춘 고대역폭메모리(HBM)와 파운드리(반도체 위탁생산), 데이터센터... 
---
제목: 이재용-올트먼-손정의 '<b>AI</b> 회동'…한미일 '<b>AI</b> 동맹' 속도내나
URL: https://n.news.naver.com/mnews/article/001/0015191319?sid=101
내용: 바로 전날 일본 도쿄에서 올트먼 CEO와 손 회장은 일본에서 합작사를 만들어 기업용 <b>생성형 AI</b>를 개발 및 판매하겠다는 계획을 밝히기도 했다. 따라서 이번 회동도 스타게이트 관련 협력 논의의 연장선에서 삼성전자가... 
---
제목: [단독]“<b>AI</b> 뒤처질 수 없다”...이해진, 네이버 이사회 의장으로 복귀
URL: https://n.news.naver.com/mnews/article/023/0003886031?sid=105
내용: 네이버는 2021년 자체 기술로 개발한 거대언어모델(LLM) 하이퍼클로바를 출시했고, 2년 뒤 기존 모델을 고도화한 <b>생성형 AI</b> 모델 하이퍼클로바X를 공개했다. 하이퍼클로바X를 한국어와 문화에 강점을 가진 토종 ‘소버린... 
---
제목: 美 스타게이트 '합종연횡'…글로벌 <b>AI</b> 선점 경쟁 뜨겁다
URL: https://n.news.naver.com/mnews/article/003/0013047908?sid=101
내용: 또 기업용 <b>생성형 AI</b> 개발, <b>생성형 AI</b> 단말기 사업도 벌인다. 전반적으로 오픈AI는 AI 본업 뿐 아니라 이를 활용한 자율주행이나 로봇, 스마트폰, TV, 가전 등에까지 영향력을 뻗

"제목: 이재용-올트먼-손정의 ‘한미일 <b>AI</b> 동맹’ 손잡았다\nURL: https://n.news.naver.com/mnews/article/020/0003613234?sid=101\n내용: 업계에서는 오픈AI와 엔비디아 등 미국 빅테크 주도의 <b>생성형 AI</b>와 AI 반도체, 일본 소프트뱅크·ARM의 자금 조달과 칩 설계 기술에 한국 기업이 갖춘 고대역폭메모리(HBM)와 파운드리(반도체 위탁생산), 데이터센터... \n---\n제목: 이재용-올트먼-손정의 '<b>AI</b> 회동'…한미일 '<b>AI</b> 동맹' 속도내나\nURL: https://n.news.naver.com/mnews/article/001/0015191319?sid=101\n내용: 바로 전날 일본 도쿄에서 올트먼 CEO와 손 회장은 일본에서 합작사를 만들어 기업용 <b>생성형 AI</b>를 개발 및 판매하겠다는 계획을 밝히기도 했다. 따라서 이번 회동도 스타게이트 관련 협력 논의의 연장선에서 삼성전자가... \n---\n제목: [단독]“<b>AI</b> 뒤처질 수 없다”...이해진, 네이버 이사회 의장으로 복귀\nURL: https://n.news.naver.com/mnews/article/023/0003886031?sid=105\n내용: 네이버는 2021년 자체 기술로 개발한 거대언어모델(LLM) 하이퍼클로바를 출시했고, 2년 뒤 기존 모델을 고도화한 <b>생성형 AI</b> 모델 하이퍼클로바X를 공개했다. 하이퍼클로바X를 한국어와 문화에 강점을 가진 토종 ‘소버린... \n---\n제목: 美 스타게이트 '합종연횡'…글로벌 <b>AI</b> 선점 경쟁 뜨겁다\nURL: https://n.news.naver.com/mnews/article/003/0013047908?sid=101\n내용: 또 기업용 <b>생성형 AI</b> 개발, <b>생성형 AI</b> 단말기 사업도 벌인다. 전반적으로 오픈AI는 AI 본업 뿐 아니라 이를 활용한 자율주행이나 로봇, 스마트폰, TV,

아래는 예시 코드입니다.

In [10]:
import requests

def get_news(query):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=30&sort=sim"
    headers = {
        'X-Naver-Client-Id': os.getenv("NAVER_CLIENT_ID"),
        'X-Naver-Client-Secret': os.getenv("NAVER_CLIENT_SECRET")
    }

    response = requests.get(url, headers=headers)
    news_json = response.json()

    result = []
    for item in news_json['items']:
        title = item['title']
        link = item['link']
        description = item['description']

        result.append(f"제목: {title}\nURL: {link}\n내용: {description}\n---")

    return "\n".join(result)

# 검색어
query = '생성형 AI'
result = get_news(query)
print(result)

제목: 이재용-올트먼-손정의 ‘한미일 <b>AI</b> 동맹’ 손잡았다
URL: https://n.news.naver.com/mnews/article/020/0003613234?sid=101
내용: 업계에서는 오픈AI와 엔비디아 등 미국 빅테크 주도의 <b>생성형 AI</b>와 AI 반도체, 일본 소프트뱅크·ARM의 자금 조달과 칩 설계 기술에 한국 기업이 갖춘 고대역폭메모리(HBM)와 파운드리(반도체 위탁생산), 데이터센터... 
---
제목: 이재용-올트먼-손정의 '<b>AI</b> 회동'…한미일 '<b>AI</b> 동맹' 속도내나
URL: https://n.news.naver.com/mnews/article/001/0015191319?sid=101
내용: 바로 전날 일본 도쿄에서 올트먼 CEO와 손 회장은 일본에서 합작사를 만들어 기업용 <b>생성형 AI</b>를 개발 및 판매하겠다는 계획을 밝히기도 했다. 따라서 이번 회동도 스타게이트 관련 협력 논의의 연장선에서 삼성전자가... 
---
제목: [단독]“<b>AI</b> 뒤처질 수 없다”...이해진, 네이버 이사회 의장으로 복귀
URL: https://n.news.naver.com/mnews/article/023/0003886031?sid=105
내용: 네이버는 2021년 자체 기술로 개발한 거대언어모델(LLM) 하이퍼클로바를 출시했고, 2년 뒤 기존 모델을 고도화한 <b>생성형 AI</b> 모델 하이퍼클로바X를 공개했다. 하이퍼클로바X를 한국어와 문화에 강점을 가진 토종 ‘소버린... 
---
제목: 美 스타게이트 '합종연횡'…글로벌 <b>AI</b> 선점 경쟁 뜨겁다
URL: https://n.news.naver.com/mnews/article/003/0013047908?sid=101
내용: 또 기업용 <b>생성형 AI</b> 개발, <b>생성형 AI</b> 단말기 사업도 벌인다. 전반적으로 오픈AI는 AI 본업 뿐 아니라 이를 활용한 자율주행이나 로봇, 스마트폰, TV, 가전 등에까지 영향력을 뻗

## 2. OpenAI chat을 이용하여 요약하기

키워드에 대한 뉴스 검색 결과를 요약합니다.   
위 결과인 `result` 데이터를 입력으로 넣은 뒤, 해당 결과에 대한 요약을 요청합니다.

In [11]:
def news_bot(messages):

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages,
        temperature = 0.1,
        max_tokens= 2048,
    )

    return response.choices[0].message.content
    # 간편한 출력을 위해 content만 추출

챗봇에 전달할 메시지는 두 개로 나눠집니다.   
위에서 얻은 get_html(query)와, 사용자가 제시할 명령입니다.   

In [12]:
query = '삼성 라이온즈'

result = get_news(query)

response = news_bot([
    {
        # f-string: f"""{변수명}"""
        "role": "system",
        "content": f"""{query}에 대한 뉴스 검색 결과가 주어집니다.
{query}에 대한 뉴스를 요약하세요.
관련 링크를 추가하세요.
---
"""
    },
    {
        "role": "user",
        "content": result
    }
])

print(response)

삼성 라이온즈 1군 선수단이 미국 괌에서의 1차 전지훈련을 성공적으로 마치고 4일 저녁 귀국했습니다. 박진만 감독은 이번 훈련에 대해 "따뜻한 날씨 속에서 선수들이 몸을 잘 만드는 데 집중했다"며 만족감을 표했습니다. 특히 신인 선수들인 배찬승과 심재훈의 준비 상태에 대해 긍정적인 평가를 내렸습니다. 삼성은 8년 만에 괌에서 스프링캠프를 진행했으며, 귀국 후 5일에는 일본 오키나와로 이동하여 2차 전지훈련을 시작할 예정입니다.

관련 링크:
- [삼성 박진만 감독 "배찬승·심재훈 등 준비 잘 돼"](https://m.sports.naver.com/kbaseball/article/001/0015191612)
- [삼성, 괌 1차 캠프 마무리...박진만 "배찬승·심재훈 준비 잘됐다"](https://m.sports.naver.com/kbaseball/article/018/0005936205)
- [괌 캠프 마친 삼성 박진만 감독 "신인들 준비 잘했다"](https://m.sports.naver.com/kbaseball/article/003/0013047022)


In [13]:
# 토큰 수 체크
import tiktoken

tokenizer  = tiktoken.encoding_for_model('gpt-4o-mini')

print("문자 수:", len(result))
print("토큰 수:", len(tokenizer.encode(result)))
print(result) #result


문자 수: 7821
토큰 수: 4615
제목: 프로야구 <b>삼성</b> 박진만 감독 &quot;배찬승·심재훈 등 준비 잘 돼&quot;
URL: https://m.sports.naver.com/kbaseball/article/001/0015191612
내용: 프로야구 <b>삼성 라이온즈</b> 1군 선수단이 미국 괌에서 1차 전지훈련을 마치고 4일 저녁 귀국한다. 박진만 삼성 감독은 이날 구단을 통해 &quot;따뜻한 날씨 속에서 트레이닝 위주로 선수들이 몸을 잘 만드는 것이 괌 전훈 캠프의... 
---
제목: 프로야구 <b>삼성</b> 박진만 감독 &quot;배찬승·심재훈 등 준비 잘 돼&quot;
URL: https://m.sports.naver.com/kbaseball/article/055/0001228666
내용: ▲ 삼성 배찬승 프로야구 <b>삼성 라이온즈</b> 1군 선수단이 미국 괌에서 1차 전지훈련을 마치고 오늘(4일) 저녁 귀국합니다. 박진만 삼성 감독은 이날 구단을 통해 &quot;따뜻한 날씨 속에서 트레이닝 위주로 선수들이 몸을 잘 만드는... 
---
제목: <b>삼성</b>, 괌 1차 캠프 마무리...박진만 &quot;배찬승·심재훈 준비 잘됐다&quot;
URL: https://m.sports.naver.com/kbaseball/article/018/0005936205
내용: <b>삼성라이온즈</b> 1군 선수단이 괌에서 1차 전지훈련을 마쳤다. <b>삼성라이온즈</b> 신인투수 배찬승. 사진=<b>삼성라이온즈</b> <b>삼성라이온즈</b> 신인 내야수 심재훈. 사진=<b>삼성라이온즈</b> 8년 만에 괌에서 스프링캠프를 치른 삼성은는 4일 1차... 
---
제목: &quot;만족스러웠다&quot; <b>삼성</b> 1군 선수단 귀국 예정…5일 日 오키나와로 출국
URL: https://m.sports.naver.com/kbaseball/article/079/0003987988
내용: 프로야구 <b>삼성 라이온즈</b> 1군 선수

다양한 명령을 넣어서 형식과 스타일을 바꿀 수 있습니다.

In [32]:
query = '기아 타이거즈'

result  = get_news(query)

result = news_bot([
    {
        "role": "user",
        "content": result
    },
    {
        "role": "user",
        "content": f"""
---
위 전체 내용을 종합하여 {query}의 동향과 미래에 대해 논쟁하는 두 사람의 대화 내용을 만들어줘.
한 명은 전문적인 말투, 한 명은 어리숙한 말투를 사용하고, 형식은 아래와 같아.
---
A:(A의 대화)
B:(B의 대화)
        """
    }
])

print(result)

A: 기아 타이거즈가 최근 스프링캠프에서 첫 불펜 투구를 시작했다는 소식을 들었나요? 조상우 선수와 함께 네일, 올러 선수도 본격적인 훈련에 돌입했다고 하니, 이번 시즌에 대한 기대가 큽니다.

B: 아, 맞아요! 조상우가 불펜에서 던졌다고 하던데, 그게 뭐 그렇게 대단한가요? 그냥 던지는 거 아닌가요?

A: 물론 던지는 것 자체는 간단해 보일 수 있지만, 스프링캠프는 선수들이 시즌을 준비하는 중요한 과정입니다. 특히 조상우 선수는 팀의 필승 카드로 기대되고 있으니, 그의 컨디션이 중요하죠.

B: 아하, 그래서 그런 거군요! 그리고 기아가 우승 기원 예·적금을 출시했다는 것도 들었어요. 그게 뭐죠?

A: 광주은행이 기아 타이거즈의 V13 우승을 기원하는 예·적금을 출시한 것입니다. 이는 지역 팬들과 함께 팀을 응원하고, 동시에 금융 상품으로서의 의미도 갖고 있습니다. 팬들의 열정이 팀에 긍정적인 영향을 미칠 수 있죠.

B: 오, 팬들이 응원하는 게 그렇게 중요하군요! 그럼 기아가 올해도 우승할 가능성이 높다고 보시나요?

A: 충분히 가능성이 있습니다. 이범호 감독이 팬들이 웃을 수 있는 야구를 하겠다고 다짐한 만큼, 팀의 분위기도 좋고, 선수들의 사기도 높습니다. 게다가 지난해 통합 우승을 차지한 경험이 큰 자산이 될 것입니다.

B: 와, 그럼 올해는 진짜 기대해도 되겠네요! 그런데 연봉 계약도 다 마무리됐다고 하던데, 선수들이 많이 올랐나요?

A: 맞습니다. 특히 김도영 선수가 역대 4년 차 최고액인 5억 원에 재계약을 했습니다. 이는 선수들의 발전과 팀의 미래를 위한 투자로 볼 수 있습니다. 기아 타이거즈가 지속적으로 강팀으로 자리 잡기 위한 기반이 마련되고 있는 셈이죠.

B: 아, 그러니까 선수들이 잘하면 팀도 잘 되고, 팬들도 행복해지는 거군요! 기아 타이거즈가 올해도 잘했으면 좋겠어요!


## 참고) 스트리밍
ChatGPT처럼 글자가 한 글자씩 순서대로 쭉 출력되도록 하면 어떨까요?   
이와 같은 작업을 스트리밍(Streaming)이라고 하며, 옵션을 추가하여 구현할 수 있습니다.

In [15]:
def news_bot_stream(messages):

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages,
        temperature = 0.1,
        max_tokens = 2048,
        stream=True # 스트리밍
    )

    return response
    # content가 아닌 전체를 return함

In [16]:
query = '거대 언어 모델'
result = get_news(query)

result = news_bot_stream([
    {
        "role": "user",
        "content": result
    },
    {
        "role": "user",
        "content": """
위 전체 내용을 종합하여 뉴스 리포팅을 해줘.
관련 링크를 추가해줘
---
'제목':
'본문':
        """
    }
])

for chunk in result:
    print(chunk.choices[0].delta.content, end='')

**제목:** AI 모델 개발 및 배포 가속화: 오라클과 업스테이지의 협력, 그리고 글로벌 AI 동향

**본문:** 최근 AI 기술의 발전이 가속화되면서 여러 기업들이 거대언어모델(LLM) 개발에 집중하고 있다. 특히, 오라클은 업스테이지에 클라우드 인프라를 공급하여 AI 모델 개발과 배포를 지원하고 있다. 업스테이지는 자체 개발한 LLM '솔라(Solar)'와 AI OCR 솔루션 '다큐먼트 AI'의 복잡한 학습 과정을 신속히 처리하기 위해 고성능 컴퓨팅 리소스를 확보하고 있다. 이로 인해 연구개발(R&D) 강화를 위한 인프라가 마련되었다.

한편, 네이버는 사우디 LEAP 전시회에 참가하여 현지 문화와 언어에 특화된 AI 개발 방향성을 제시하고 있다. 네이버는 자사의 LLM을 기반으로 사우디의 언어적, 문화적 특성을 존중하는 AI 솔루션을 개발하고 있으며, 이는 AI 밸류체인의 실제 구현 사례로 주목받고 있다.

또한, 솔트룩스는 초거대 언어모델 '루시아2.5'를 공개하며 법률, 의료, 제조, 교육 등 다양한 분야에서 높은 성능을 발휘할 수 있다고 밝혔다. 이 모델은 챗GPT 대비 최대 20분의 1 수준의 비용으로 운영할 수 있어 주목받고 있다.

AI 모델의 경쟁이 치열해짐에 따라, 카카오는 오픈AI의 기술을 활용하기로 결정했으며, 엔비디아는 자사의 GPU를 통해 AI 모델의 배포를 지원하고 있다. 이러한 흐름 속에서 중국의 AI 모델 '딥시크'가 미국의 기술기업들과 경쟁하고 있는 상황도 주목할 만하다.

AI 기술의 발전은 다양한 산업에 걸쳐 혁신을 가져오고 있으며, 앞으로의 동향이 더욱 기대된다.

관련 링크:
- [오라클, 업스테이지에 OCI 공급... AI 모델 개발·배포 가속화](https://n.news.naver.com/mnews/article/366/0001051275?sid=105)
- [네이버, 사우디 LEAP 참가…현지 문화·언어에 특화된 '소버린 AI'강조](https://n.news.naver.com/mnews/article/003

이와 같은 스트리밍은 LLM 어플리케이션의 마지막 출력에서 사용하면    
사용자 경험을 향상시킬 수 있습니다.

<br><br><br>
## Tool(과거 Function Call) : 커스텀 함수 사용하기
만약 query를 명시적으로 지정하지 않고, 사용자의 질문에 따라 관련 query를 추출하고,  
이를 통해 검색하게 할 수도 있을까요?    

`Tool Calling` 를 추가하면 이를 수행할 수 있습니다.    

Web_Search라는 이름의 툴을 만들고, 이를 `get_html`과 연결하겠습니다.

-------


In [17]:
from pydantic import BaseModel, Field

class Web_Search(BaseModel):
    """query를 이용하여 뉴스 검색"""
    query: str= Field(description="""검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "개봉영화", "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"
        """)

tools = [openai.pydantic_function_tool(Web_Search)]
tools

[{'type': 'function',
  'function': {'name': 'Web_Search',
   'strict': True,
   'parameters': {'description': 'query를 이용하여 뉴스 검색',
    'properties': {'query': {'description': '검색 키워드\n규칙:\n1. 최대 2개 단어로 구성\n2. 불필요한 조사나 형용사 제외\n3. 핵심 명사만 포함\n\n예시:\n- (좋음) "개봉영화", "영화"\n- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"\n        ',
      'title': 'Query',
      'type': 'string'}},
    'required': ['query'],
    'title': 'Web_Search',
    'type': 'object',
    'additionalProperties': False},
   'description': 'query를 이용하여 뉴스 검색'}}]

Tool 정보가 포함된 gpt는 함수의 구조와 설명을 이용하여 어떤 함수를 써야 하는지를 판단합니다.

직접적으로 실행하는 것은 아니지만, 결과를 활용하면 해당 함수를 바로 사용할 수 있습니다.

In [18]:
def news_bot_v2(messages, stream=False, model = 'gpt-4o-mini'):

    # 사용할 툴 펑션의 목록과 설명을 리스트로 전달
    # LLM이 스스로 description, name, parameter의 값을 통해 판단
    class Web_Search(BaseModel):
        """query를 이용하여 뉴스 검색"""
        query: str= Field(description="""
검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"
""")

    tools = [openai.pydantic_function_tool(Web_Search)]

    response = client.chat.completions.create(
        model = model,
        messages = messages,

        tools = tools, # 사용할 툴 목록 전달
        tool_choice = 'auto',
        # 'none'이면 툴 사용하지 않음
        # 'required'면 무조건 툴 사용

        # parallel_tool_calls = False,
        # 툴 1회만 출력하기

        temperature= 0.1,
        max_tokens= 1024,

        stream = stream
        # stream = True 면 스트리밍, 아니면 기본 출력
    )

    # Streaming 여부에 따라 출력 다르게 하기
    if stream: return response

    return response.choices[0].message

In [19]:
# tool 사용 필요 없음
result = news_bot_v2([
    {
        "role": "user",
        "content": """안녕하세요! 오늘 날씨가 좋네요."""
    }
])

result

ChatCompletionMessage(content='안녕하세요! 날씨가 좋다니 기쁘네요. 오늘 어떤 계획이 있으신가요?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [20]:
# tool 사용 필요함

tool_call_result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요?
"""
    }
])

print(tool_call_result)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Pfp6fz80ra5fEDlo8neSlvRb', function=Function(arguments='{"query":"영화"}', name='Web_Search'), type='function')])


Tool Call 메시지는 content 대신 tool_calls로 전달됩니다.

In [21]:
tool_call_result.tool_calls

[ChatCompletionMessageToolCall(id='call_Pfp6fz80ra5fEDlo8neSlvRb', function=Function(arguments='{"query":"영화"}', name='Web_Search'), type='function')]

In [22]:
(tool_call_result.tool_calls[0].id,
tool_call_result.tool_calls[0].function.name,
tool_call_result.tool_calls[0].function.arguments)

('call_Pfp6fz80ra5fEDlo8neSlvRb', 'Web_Search', '{"query":"영화"}')

LLM이 스스로 판단하여, 실행할 함수의 정보를 `tool_calls`에 돌려줍니다.

tool_calls의 구성 요소는 다음과 같습니다.
- id: tool call의 id로, 해당 id에 실행 결과를 연결할 수 있습니다.
- function : arguments와 name을 통해 실행할 툴의 이름과 매개변수를 전달합니다.
<br><br>


tool_calls의 결과를 임의로 포함하여 전달해 보겠습니다.   
`tool`타입의 메시지로 전달해야 합니다.

In [23]:
result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요?
"""
    },
    tool_call_result, # tool_calls 메시지
    {
        "role": "tool",
        "content": """
청설, 보통의 가족, 아노라, 아마존 활명수, 베놈:라스트 댄스
""",
        "tool_call_id":tool_call_result.tool_calls[0].id
    }
])

result

ChatCompletionMessage(content='요즘 새로 개봉한 영화로는 다음과 같은 작품들이 있습니다:\n\n1. 청설\n2. 보통의 가족\n3. 아노라\n4. 아마존 활명수\n5. 베놈: 라스트 댄스', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

이번에는 실제 함수를 실행하여 전달해 보겠습니다.

In [24]:
tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments

('Web_Search', '{"query":"영화"}')

함수명과 함수에 전달할 매개변수값이 주어지면, 아래와 같이 함수를 불러올 수 있습니다.

In [25]:
# 문자열을 Dict로 바꾸기
import json

example = '{"query":"LLM"}'
example_dict = json.loads(example)
# json으로 변환

print(example_dict)
print(type(example_dict))



{'query': 'LLM'}
<class 'dict'>


In [26]:
# 함수이름 문자열을 함수로 바꾸기

available_functions = {'Web_Search': get_news}

available_functions['Web_Search']

<function __main__.get_news(query)>

In [27]:
# Dictionary를 함수의 매개변수로 전달하기

def f(a,b):
  return a + b

example2 = {'a': 1, 'b': 2}

f(**example2)


3

In [28]:
available_functions['Web_Search'](**example_dict)

"제목: <b>LLM</b> 보안 전문 솔루션 필요할까?···87%가 &quot;예스&quot;\nURL: https://n.news.naver.com/mnews/article/092/0002361970?sid=105\n내용: 이스트시큐리티(대표 정진일)는 지난 12월 개최한 기업의 <b>LLM</b> 필수 보안 전략 웨비나에서 국내 기업 보안 담당자 및 실무자 200여 명을 대상으로 설문조사를 진행하고 그 결과를 5일 발표했다. 그 결과, <b>LLM</b> 보안... \n---\n제목: 코난테크놀로지, 생성형 AI 기반 의료 <b>LLM</b> 선보인다\nURL: https://n.news.naver.com/mnews/article/030/0003281228?sid=105\n내용: 인공지능(AI) 소프트웨어 기업 코난테크놀로지가 생성형 AI 기술을 기반으로 의료 분야에 특화된 거대언어모델(<b>LLM</b>)을 선보인다. 코난테크놀로지는 한림대의료원과 '생성형 AI 기반 입원환자 전주기 기록지 작성·의료원... \n---\n제목: 딥시크 추격에…오픈AI 올트먼, 한국서 주도권다지기 '광폭행보'(종합)\nURL: https://n.news.naver.com/mnews/article/001/0015191966?sid=105\n내용: 오픈AI의 미국 내 최대 경쟁자인 구글과 마이크로소프트(MS)는 대형언어모델(<b>LLM</b>)뿐만 아니라 기기에... 카카오 &quot;오픈AI와 서비스 공동 개발&quot; <b>LLM</b> 자체 개발서 협력으로 선회 지난해 말 인공지능(AI) 서비스 '카나나'를... \n---\n제목: 올트먼 만난 김창한 크래프톤 대표 &quot;게임 특화 AI 논의&quot;\nURL: https://n.news.naver.com/mnews/article/003/0013046577?sid=105\n내용: 거대언어모델(<b>LLM</b>)을 기반으로 한 CPC(Co-Playable Character: 협동 플레이 가능한 캐릭터) 개발과 게

위 과정을 조합하여 실행해 보겠습니다.

In [29]:
available_functions = {'Web_Search':get_news}

name, arguments = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments


result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요? 오늘 날짜는 24년 11월 18일입니다.
"""
    },
    tool_call_result, # tool_calls 메시지
    {
        "role": "tool",
        "content": available_functions[name](**json.loads(arguments)),  
        # get_news(query = '영화')

        "tool_call_id":tool_call_result.tool_calls[0].id
    }
])

result.content

"최근 개봉한 영화 중 하나는 **'레드벨벳 해피니스 다이어리: 마이 디어, 레베럽 인 시네마'**입니다. 이 영화는 레드벨벳의 데뷔 10주년 기념 팬콘서트 실황을 담고 있으며, 11월 5일에 개봉했습니다. \n\n또한, **'캡틴 아메리카: 브레이브 뉴 월드'**라는 영화도 개봉을 앞두고 있으며, 이 영화는 한국 영화 '달콤한 인생'에서 영감을 받은 액션 장면이 포함되어 있다고 합니다. \n\n더 많은 정보가 필요하시면 말씀해 주세요!"

스트리밍 기능을 추가하고, 전체 과정을 함수로 자동화합니다.:

In [30]:
def news_qa(prompt):

    print('Prompt:',prompt)
    # 출력 결과를 함수와 연결하기 위한 dictionary
    available_functions = {'Web_Search':get_news}

    tool_call_result = news_bot_v2([
        {
            "role": "user",
            "content": prompt
        }
    ])
    # 프롬프트를 받아 Web_Search를 수행할지 결정합니다.
    # 단순 대화라면 tool call을 생성하지 않습니다.

    print('---')
    print('News_Bot: Call ', end='')

    if tool_call_result.tool_calls: # tool_calls가 존재하면:

        name, arguments = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments

        print(name,arguments)

        # get_news(query)
        result = available_functions[name](**json.loads(arguments))
        # 함수 선택, 매개변수 전달
        print('---')
        print('News_Bot:',end='')

        response = news_bot_v2(
        [
        {
            'role':"system",
            'content':'사용자의 질문과 이를 답변하기 위한 정보가 주어집니다. 이를 바탕으로 답변하세요.'
        },

        {
            "role": "user",
            "content": prompt
        },
        tool_call_result, # tool_calls 메시지
        {
            "role": "tool",
            "content": result,
            "tool_call_id":tool_call_result.tool_calls[0].id
        }],
        stream=True
        )
        for chunk in response:
            print(chunk.choices[0].delta.content, end='')
    else:
        print('Nothing')
        print(tool_call_result.content)

# example
# "오늘 가장 큰 뉴스는 뭐야?"
# """오픈AI는 요즘 새로운 소식 있어?"""
prompt = """넷플릭스 신작 추천해줘."""

news_qa(prompt)

Prompt: 넷플릭스 신작 추천해줘.
---
News_Bot: Call Web_Search {"query":"넷플릭스 신작"}
---
News_Bot:최근 넷플릭스에서 공개될 신작 중 몇 가지를 추천해 드립니다:

1. **계시록** - 연상호 감독이 제작한 이 작품은 알폰소 쿠아론 감독이 이그제큐티브 프로듀서로 참여한 영화입니다. 다양한 장르의 요소가 결합된 스릴러로 기대를 모으고 있습니다.

2. **이 별에 필요한** - 넷플릭스의 첫 번째 한국 애니메이션 영화로, 독특한 스토리와 비주얼로 많은 관심을 받고 있습니다.

3. **동미새: 동호회에 미친 새내기** - '솔로지옥' 제작진이 만든 신작 예능으로, 매주 월요일마다 새로운 에피소드가 공개됩니다.

4. **흑백요리사: 요리 계급 전쟁** - 요리 서바이벌 형식의 예능으로, 다양한 요리 대결을 통해 시청자들에게 재미를 선사할 예정입니다.

5. **오징어 게임 시즌 3** - 전 세계적으로 큰 인기를 끌었던 '오징어 게임'의 새로운 시즌이 곧 공개됩니다.

이 외에도 다양한 장르의 신작들이 준비되어 있으니, 기대해 보세요!None

In [31]:
news_qa("회사 가기 싫어.")

Prompt: 회사 가기 싫어.
---
News_Bot: Call Nothing
회사가 가기 싫은 기분이 드는 건 자연스러운 일이에요. 어떤 이유로 그런 기분이 드는지 이야기해볼까요? 스트레스, 업무량, 동료 관계 등 여러 가지 이유가 있을 수 있어요.


# [심화] Multiple Tool Call
때로는, 하나의 메시지에서 여러 개의 Tool Call을 수행해야 하기도 합니다.   


반복문을 통해, Tool Call의 목록을 한꺼번에 전달합니다.

In [41]:
def news_qa_v2(prompt, model = 'gpt-4o-mini'):

    print('Prompt:',prompt)
    # 출력 결과를 함수와 연결하기 위한 dictionary
    available_functions = {'Web_Search':get_news}

    tool_call_result = news_bot_v2([
        {
            "role": "user",
            "content": prompt
        },
    ])
    # 프롬프트를 받아 Web_Search를 수행할지 결정합니다.
    # 단순 대화라면 tool call을 생성하지 않습니다.

    print('---')
    print('News_Bot: Call ', end='')

    if tool_call_result.tool_calls: # tool_calls가 존재하면:



        ### Tool Message 복수 입력

        tool_messages=[]
        for tool_call in tool_call_result.tool_calls:
            name, arguments = tool_call.function.name, tool_call.function.arguments
            print(name,arguments)

            # get_news(query)
            result = available_functions[name](**json.loads(arguments))
            # 함수 선택, 매개변수 전달
            print('---')
            print('News_Bot:',end='')
            tool_messages.append(
                {"role": "tool","content": result,"tool_call_id":tool_call.id}
            )
        ####



        response = news_bot_v2(
        [
        {
            'role':"system",
            'content':'사용자의 질문과 이를 답변하기 위한 정보가 tool 실행 결과를 통해 주어집니다. 전체 내용을 최대한 자세히 읽고, 상세하게 답변하세요.'
        },

        {
            "role": "user",
            "content": prompt
        },
        tool_call_result] + tool_messages,
        stream=True,
        model = model
        )
        for chunk in response:
            print(chunk.choices[0].delta.content, end='')
    else:
        print('Nothing')
        print(tool_call_result.content)


prompt = """넷플릭스 왓챠 디즈니플러스 웨이브 신작 추천해줘."""

news_qa_v2(prompt)

Prompt: 넷플릭스 왓챠 디즈니플러스 웨이브 신작 추천해줘.
---
News_Bot: Call Web_Search {"query": "넷플릭스 신작"}
---
News_Bot:Web_Search {"query": "왓챠 신작"}
---
News_Bot:Web_Search {"query": "디즈니플러스 신작"}
---
News_Bot:Web_Search {"query": "웨이브 신작"}
---
News_Bot:다양한 OTT 플랫폼에서의 신작 추천을 위해 최근 뉴스 기사를 참고했습니다. 아래는 넷플릭스, 왓챠, 디즈니플러스, 웨이브에서 추천할 만한 신작입니다.

### 넷플릭스
1. **신작: 'Mr. 포트너'**
   - **장르**: 드라마
   - **내용**: 이 드라마는 주인공이 다양한 사건을 통해 성장하는 이야기를 담고 있습니다. 11월 14일에 공개됩니다.
   - **링크**: [기사 링크](https://m.entertain.naver.com/article/311/0001797192)

2. **신작: '좀비버스: 뉴 블러드'**
   - **장르**: 리얼리티, 좀비
   - **내용**: K-좀비를 테마로 한 리얼리티 쇼로, 참가자들이 좀비 상황에서 생존하는 과정을 그립니다. 11월 19일에 공개됩니다.
   - **링크**: [기사 링크](https://sports.hankooki.com/news/articleView.html?idxno=6881142)

### 왓챠
1. **신작: '신작'**
   - **장르**: 다양한 장르
   - **내용**: 왓챠에서 다양한 신작들이 공개되고 있으며, 특히 한국 드라마와 영화가 주목받고 있습니다. 구체적인 제목은 기사에서 확인할 수 있습니다.
   - **링크**: [기사 링크](https://m.entertain.naver.com/article/609/0000922147)

### 디즈니플러스
1. **신작: '가디언즈 오브 갤럭시 Vol. 3'**
   - **장르**: 액션, 어드벤처


In [42]:
news_qa_v2(prompt, model='gpt-4o')

Prompt: 넷플릭스 왓챠 디즈니플러스 웨이브 신작 추천해줘.
---
News_Bot: Call Web_Search {"query": "넷플릭스 신작"}
---
News_Bot:Web_Search {"query": "왓챠 신작"}
---
News_Bot:Web_Search {"query": "디즈니플러스 신작"}
---
News_Bot:Web_Search {"query": "웨이브 신작"}
---
News_Bot:최근 각 스트리밍 플랫폼에서 공개된 신작들을 소개합니다.

### 넷플릭스
1. **좀비버스: 뉴 블러드** - K-좀비 예능으로, 좀비와의 생존을 다루는 콘텐츠입니다.
2. **미키 17** - 북미 극장에서 개봉 예정인 영화로, 넷플릭스 시리즈 '성난 사람들'과 함께 주목받고 있습니다.
3. **사흘** - 박신양 주연의 드라마로, 복잡한 인간관계를 다루고 있습니다.

### 왓챠
1. **유마 카운티의 끝에서: 주유소 살인사건** - 범죄 스릴러 영화로, 긴장감 넘치는 스토리를 제공합니다.
2. **러브 라이즈 블리딩** - 로맨스 드라마로, 사랑과 갈등을 그립니다.
3. **개소리** - 코미디 드라마로, 일상 속의 유머를 담고 있습니다.

### 디즈니플러스
1. **강남 비-사이드** - 한국 드라마로, 강남의 화려한 삶을 배경으로 한 이야기를 다룹니다.
2. **조명가게** - 미스터리 드라마로, 조명가게에서 벌어지는 사건들을 중심으로 전개됩니다.
3. **왕과 사는 남자** - 역사 드라마로, 조선시대를 배경으로 한 이야기입니다.

### 웨이브
1. **미안하다, 사랑한다 2024** - 클래식 드라마의 리메이크로, 현대적인 감각을 더했습니다.
2. **여왕벌 게임** - 서바이벌 게임을 소재로 한 드라마로, 긴장감 넘치는 전개가 특징입니다.
3. **내 이름은 김삼순 2024** - 인기 드라마의 리메이크로, 새로운 캐스팅과 함께 돌아왔습니다.

각 플랫폼에서 다양한 장르의 신작들이 공개되고 있으니, 관심 있는 작품을 찾아보세요!N